In [27]:
%load_ext autoreload
%autoreload 2

import h5py
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit

mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
import util as yu
yu.flag_fast=True

path='dat/NST_b_cD96.h5'
with h5py.File(path) as f:
    t=(f['diags/N/data/N2_N2'][:,:,0]+f['diags/N/data/N2_N2'][:,:,1])/2
    C2pt_dat=np.real(t)
    
    print(f.keys())
    print(f['diags/'].keys())
    print(f['diags/NJN'].keys())
    print(f['diags/NJN/data'].keys())
    print(f['diags/NJN/data/gS_j+_deltat_10'])
    tfs=[int(ele.split('_')[-1]) for ele in f['diags/NJN/data'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    print(tfs)
    
    C3pt_dat_dic={tf:np.real(f['diags/NJN/data/gS_j+_deltat_{}'.format(tf)][:]) for tf in tfs}
    print(C3pt_dat_dic[12].shape)
        
    print()
    print(f['diags/N-j'].keys())
    print(f['diags/N-j/opabs'][:])
    print(f['diags/N-j/inserts'][:])
    print(f['diags/N-j/data'].keys())
    print(f['diags/N-j/data/N2_j+_N2_deltat_10'])
    
    tfs_disc=[int(ele.split('_')[-1]) for ele in f['diags/N-j/data'].keys()]
    tfs_disc=list(set(tfs_disc))
    tfs_disc.sort()
    C3pt_disc_dat_dic={tf:np.real((f['diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)][:,:,0,0]+f['diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)][:,:,1,0])/2) for tf in tfs_disc}

    print()
    print(f['VEV'].keys())
    print(f['VEV/j'].keys())
    print(f['VEV/j/inserts'][:])
    print(f['VEV/j/data'].keys())
    print(f['VEV/j/data/j+'])
    vev_dat=f['VEV/j/data/j+'][:,0]

aMUL=0.00054
ZP=0.493
aLat=0.05692 # lattice spacing a in fm
aInvLat=1/aLat*197.3/1000 # a^{-1} in GeV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
<KeysViewHDF5 ['VEV', 'cfgs', 'diags']>
<KeysViewHDF5 ['N', 'N-j', 'NJN', 'N_correlated_NJN']>
<KeysViewHDF5 ['data']>
<KeysViewHDF5 ['gS_j+_deltat_10', 'gS_j+_deltat_12', 'gS_j+_deltat_14', 'gS_j+_deltat_16', 'gS_j+_deltat_18', 'gS_j+_deltat_20', 'gS_j+_deltat_22', 'gS_j+_deltat_24', 'gS_j+_deltat_26', 'gS_j+_deltat_8']>
<HDF5 dataset "gS_j+_deltat_10": shape (494, 11), type "<f4">
[8, 10, 12, 14, 16, 18, 20, 22, 24, 26]
(494, 13)

<KeysViewHDF5 ['data', 'inserts', 'opabs']>
[b'g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
[b'id']
<KeysViewHDF5 ['N2_j+_N2_deltat_10', 'N2_j+_N2_deltat_11', 'N2_j+_N2_deltat_12', 'N2_j+_N2_deltat_13', 'N2_j+_N2_deltat_14', 'N2_j+_N2_deltat_15', 'N2_j+_N2_deltat_16', 'N2_j+_N2_deltat_17', 'N2_j+_N2_deltat_18', 'N2_j+_N2_deltat_19', 'N2_j+_N2_deltat_20', 'N2_j+_N2_deltat_21', 'N2_j+_N2_deltat_22', 'N2_j+_N2_deltat_23', 'N2_j+_N2_delta